<a href="https://colab.research.google.com/github/MohamedElgharbawy/Z3Performance/blob/main/Z3_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plotting Z3 Performance

This program plots Z3's performance when solving theorems with n random variables and a random set of [1, n<sup>2</sup>] constraints.

In [1]:
!pip3 install z3-solver

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from z3 import *
import random
import operator
import time
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [3]:
ops = [operator.lt, operator.le, operator.ge, operator.gt]

In [4]:
def z3_performance(num_var: int, use_max: bool, inc_constraints=1):
  """
  Measure Z3's performance to check a model for num_var variables with
  1 - num_var**2 constraints.
  Returns the a tuple, a list of the number of constraints, and a list of the
  corresponding times.
  """
  X = IntVector('x', num_var)

  if use_max:
    solver = Optimize()
    solver.maximize(Sum(X))
  else:
    solver = Solver()

  # Graphing lists
  num_constraints = []
  times = []
  num_unsat = 0

  # Constraints
  constraints = []
  for i in range(1, num_var**2 + 1, inc_constraints):
    print(f"{num_var}: {i} constraints")
    num_constraints.append(i)

    # Get a random operator
    op = random.choice(ops)
    
    # Create a random constraint
    constraint = op(sum(random.randint(-10, 10)*X[j] for j in range(0,num_var)), random.randint(-10, 10))
    constraints.append(constraint)
    solver.add(constraint)

    # Check model
    start_time = time.monotonic()
    satisfiability = str(solver.check())
    final_time = time.monotonic() - start_time
    times.append(round(final_time, 5))

    if satisfiability == "unsat":
      num_unsat += 1
    
    if num_unsat == 10:
      break

  return num_constraints, times

In [5]:
def plot_one_z3_performance(num_var: int, use_max: bool, inc_constraints=1):
  """
  Measure and plot Z3's performance for num_var variables
  """
  num_constraints, times = z3_performance(num_var, use_max, inc_constraints)

  plt.plot(num_constraints, times)

  plt.title("Z3 Performance for {} variables".format(num_var))
  plt.xlabel("Number of constraints")
  plt.ylabel("Time (seconds)")
  plt.savefig("plots/one/one_{},use_max={}.png".format(num_var, use_max))
  plt.clf()

In [6]:
def plot_all_z3_performance(num_var: int, use_max: bool):
  """
  Measure and plot Z3's performance for [1, num_var] variables
  """
  for i in range(1, num_var + 1):
    print("Random variable: {}".format(i))
    num_constraints, times = z3_performance(i, use_max)

    plt.plot(num_constraints, times, label=str(i))

  plt.title("Z3 Performance")
  plt.xlabel("Number of constraints")
  plt.ylabel("Time (seconds)")
  plt.legend(title="Number of variables")
  plt.savefig(f"plots/all/all_{i},use_max={use_max}")
  plt.clf()

In [7]:
random.seed(42)
plot_one_z3_performance(25, use_max=True, inc_constraints=5)
plot_one_z3_performance(25, use_max=False, inc_constraints=5)

25: 1 constraints
25: 6 constraints
25: 11 constraints
25: 16 constraints
25: 21 constraints
25: 26 constraints
25: 31 constraints
25: 36 constraints
25: 41 constraints
25: 46 constraints
25: 51 constraints


In [ ]:
random.seed(42)
plot_one_z3_performance(50, use_max=True, inc_constraints=10)
plot_one_z3_performance(50, use_max=False, inc_constraints=10)